In [1]:
import numpy as np
import pandas as pd

In [2]:
#create example 
rnd = np.random
rnd.seed(5)
n = 20 +1
xc = rnd.rand(n+1)*100
yc = rnd.rand(n+1)*100

customer_list = [] #[Customernr, x-Koordinate, y-Koordinate, order_quantity]
for i in range(1, n+1):
    xc[i] = xc[i]-xc[0]
    yc[i] = yc[i]-yc[0]
    
xc = np.delete(xc, 0)
yc = np.delete(yc, 0)

for i in range(0, n):
    customer_list.append([i+1, xc[i] ,yc[i] , np.random.uniform(0,10)] )


In [3]:
print(customer_list)

[[1, 64.87391350876369, -3.1097674867576295, 0.022142125324435824], [2, -1.5274015750313055, 4.287806704737026, 4.694883717575773], [3, 69.66177368481821, -14.052157998794335, 9.814687376060105], [4, 26.641801770508966, -11.907301944009618, 3.9894480390285048], [5, 38.97506918129062, 67.9244648426578, 8.137324775869182], [6, 54.39146853905761, 67.55408346443299, 5.464564979724777], [7, 29.642481678320376, -9.627122504542864, 7.708540871402246], [8, 7.480733048648247, -26.037931901187683, 4.84931074988458], [9, -3.4271942428487847, -8.013033426141316, 0.2911156367271772], [10, -14.12519023248646, 41.51577334851936, 0.8652568840220509], [11, 51.64471251092306, 49.48287049141659, 1.1145381237802332], [12, 21.931605180621364, -26.175278820228232, 2.5124511169054764], [13, -6.3683303377088265, 29.297697748839223, 9.64915292526365], [14, 65.79438601115395, -28.304370792531852, 6.317660527377455], [15, 5.2093290902485165, 23.078673126402972, 8.166602026153619], [16, 19.224184799131184, 35.510

In [4]:
def sweep_algorithm(customer_list, max_capacity):
    #the zero point of the coordinate system used in is customer_list has warehouse as the centre
    warehouse = [0,0,0,0, None]
    
    #customer_list = [Customernr, x-coordinate, y-coordinate, order_quantity] ...
    if len(customer_list) == 0:
        return None, None, "No element is in given List."
    else:
        for i in customer_list:
            if len(i) != 4:
                return None, None, "The given 2D List needs the entrys [Customernr, x-coordinate, y-coordinate, order_quantity]"
    
    #max_capacity = number
    if isinstance(max_capacity, (int, float))== False:
        return None, None, "The given maximal Capacity is not a number."
    
    
    
    #list check types of row 
    for i in customer_list:
        if isinstance(i[1], (int, float)) and isinstance(i[2], (int, float)) and isinstance(i[3], (int, float)):
            #check postive delivery quantity
            if i[3] < 0:
                return None, "A negative delivery quanitity is not possible."
            elif i[3] > max_capacity:
                return None, None, "Customer " + str(i[0]) + " orderd more than the max capacity."
        else:
            return None, None, "The types of the list need to be [free_selectable, number, number, number]"
 
   
    
    capacity = 0
    cluster_counter = 0
    cluster_list = [[]] # list of lists of customers
    
    for customer in reversed(customer_list):
        #1. Quadrant
        if customer[1] > 0 and customer[2] > 0:
            customer.append(np.rad2deg(np.arctan(customer[2]/customer[1])))
        #2. Quadrant
        elif customer[1] < 0 and customer[2] > 0:
            customer.append(np.rad2deg(np.arctan(customer[2]/customer[1]))+180)
        #3. Quadrant
        elif customer[1] < 0 and customer[2] < 0:
            customer.append(np.rad2deg(np.arctan(customer[2]/customer[1]))+180)
        #4 Quadrant
        elif customer[1] > 0 and customer[2] < 0:
            customer.append(np.rad2deg(np.arctan(customer[2]/customer[1]))+360)
        #Ausnahmen direkt auf Achsen 
        elif customer[1] == 0 :
            if customer[2] > 0:
                customer.append(90)
            elif customer[2] < 0:
                customer.append(270)
            elif customer[2] == 0:
                customer_list.remove(customer)
        elif customer[2] == 0 and customer[1] != 0:    
            if customer[1] > 0:
                customer.append(0)
            elif customer[1] < 0:
                customer.append(180)

    customer_list = sorted(customer_list, key=lambda x:x[4])


    
    while len(customer_list) != 0:
        if capacity + customer_list[0][3] <= max_capacity:
            capacity = capacity + customer_list[0][3]
            cluster_list[cluster_counter].append(customer_list.pop(0))  
        elif capacity + customer_list[0][3] >= max_capacity: 
            capacity = 0
            cluster_counter = cluster_counter +1
            cluster_list.append([])
            
    return cluster_list, warehouse, None

        

cluster_list, warehouse, error = sweep_algorithm(customer_list, 30)     

In [6]:
df_list = []
if error == None:
    for i in cluster_list:
        df_list.append(pd.DataFrame(i, columns = ["Nr.", "x", "y", "amount", "degree"]))
        display(df_list[-1])
else:
    print(error)

,Nr.,x,y,amount,degree
0,11,51.644713,49.482870,1.114538,43.775353
1,6,54.391469,67.554083,5.464565,51.160567
2,19,35.784464,51.781100,9.266826,55.352738
3,20,37.793603,58.579721,9.126268,57.171296


,Nr.,x,y,amount,degree
0,5,38.975069,67.924465,8.137325,60.152789
1,16,19.224185,35.510930,5.660820,61.570673
2,15,5.209329,23.078673,8.166602,77.280331
3,17,7.408676,70.093852,6.353562,83.966438


,Nr.,x,y,amount,degree
0,21,4.382595,63.806373,8.248107,86.070762
1,13,-6.368330,29.297698,9.649153,102.263402
2,10,-14.125190,41.515773,0.865257,108.790190
3,2,-1.527402,4.287807,4.694884,109.606905
4,9,-3.427194,-8.013033,0.291116,246.843484
5,8,7.480733,-26.037932,4.849311,286.029430


,Nr.,x,y,amount,degree
0,12,21.931605,-26.175279,2.512451,309.958792
1,4,26.641802,-11.907302,3.989448,335.918195
2,14,65.794386,-28.304371,6.317661,336.722900
3,7,29.642482,-9.627123,7.708541,342.007524


,Nr.,x,y,amount,degree
0,3,69.661774,-14.052158,9.814687,348.595352
1,18,40.679474,-2.558828,8.119024,356.400711
2,1,64.873914,-3.109767,0.022142,357.255595


In [7]:
from math import sqrt
import copy 

def nearest_insertion(cluster_list):
    Distance_List = []
    Cluster_Distance_Lists = []
    List_counter = 0
    Tour_List = []
    for cluster in cluster_list:    
        cluster.insert(0, warehouse)
        for customer_row in cluster:
            Distance_List.append([])

            #Distance_List[List_counter].append(np.nan)
            for customer_column in cluster:
                Pythagoras = sqrt(((customer_row[1]-customer_column[1])**2) + ((customer_row[2]-customer_column[2])**2))
                if Pythagoras == 0:
                    Distance_List[List_counter].append(np.nan)
                elif Pythagoras != 0:
                    Distance_List[List_counter].append(Pythagoras)

            List_counter += 1

        Cluster_Distance_Lists.append(Distance_List)
        List_counter = 0
        Distance_List = []

    #two selected customer (Start of Nearest_Insertion)
    cluster_list_copy = copy.deepcopy(cluster_list)

    for cluster_nr in range(0, len(cluster_list)):
            Distance_List = np.array(Cluster_Distance_Lists[cluster_nr])
            Distance_List_copy = np.copy(Distance_List)
            Distance_List[:,0] = np.nan

            Tour = [cluster_list_copy[cluster_nr].pop(0), cluster_list[cluster_nr][np.nanargmin(Distance_List[0])]]
            cluster_list_copy[cluster_nr].remove(cluster_list[cluster_nr][np.nanargmin(Distance_List[0])])
            Index_Tour_Distance = [0, np.nanargmin(Distance_List[0])]

            Distance_List[:,np.nanargmin(Distance_List[0])] = np.nan



            while len(cluster_list_copy[cluster_nr]) != 0:
                min_Distance = 9999

                #select nearest customer to already selected stops
                for index in Index_Tour_Distance:
                    min_row = np.nanmin(Distance_List[index,:])

                    if min_Distance > min_row: 
                        min_Distance = min_row
                        added_customer_index = np.nanargmin(Distance_List[index,:])
                        index_closest_to = index

                #if not last element get the next element to compare
                if  Index_Tour_Distance.index(index_closest_to)+1 <=  len(Index_Tour_Distance)-1:
                    after_closest_index = Index_Tour_Distance[Index_Tour_Distance.index(index_closest_to)+1] 
                    after_closest = Distance_List[after_closest_index, added_customer_index]
                elif Index_Tour_Distance.index(index_closest_to) ==  len(Index_Tour_Distance)-1:
                    after_closest_index = Index_Tour_Distance[0] 
                    after_closest = Distance_List[after_closest_index, added_customer_index]


                #if not first element compare with the elment before this one
                if Index_Tour_Distance.index(index_closest_to) > 0 :
                    before_closest_index = Index_Tour_Distance[Index_Tour_Distance.index(index_closest_to)-1] 
                    before_closest = Distance_List[before_closest_index, added_customer_index]
                elif Index_Tour_Distance.index(index_closest_to) == 0:
                    before_closest_index = Index_Tour_Distance[-1]
                    before_closest = Distance_List[before_closest_index, added_customer_index]


                #select shortest insertion 
                
                c_before =  Distance_List[before_closest_index, added_customer_index] + \
                            Distance_List[index_closest_to, added_customer_index]- \
                            Distance_List_copy[before_closest_index, index_closest_to] 
                c_after =   Distance_List[after_closest_index, added_customer_index] + \
                            Distance_List[index_closest_to, added_customer_index] - \
                            Distance_List_copy[after_closest_index, index_closest_to]
                

                if c_before <= c_after: 
                    if index_closest_to == 0:
                        Index_Tour_Distance.append(added_customer_index)
                        Tour.append(cluster_list[cluster_nr][added_customer_index])
                    else:
                        temp = Index_Tour_Distance.index(index_closest_to)
                        Index_Tour_Distance.insert(temp , added_customer_index)
                        Tour.insert(temp, cluster_list[cluster_nr][added_customer_index])
                    Distance_List[:, added_customer_index] = np.nan
                    cluster_list_copy[cluster_nr].remove(cluster_list[cluster_nr][added_customer_index])
                elif c_after < c_before:
                    if after_closest_index == 0:
                        Index_Tour_Distance.append(added_customer_index)
                        Tour.append(cluster_list[cluster_nr][added_customer_index])
                    else:
                        temp = Index_Tour_Distance.index(after_closest_index)
                        Index_Tour_Distance.insert(temp, added_customer_index)
                        Tour.insert(temp, cluster_list[cluster_nr][added_customer_index])

                    Distance_List[:, added_customer_index] = np.nan
                    cluster_list_copy[cluster_nr].remove(cluster_list[cluster_nr][added_customer_index])

            Tour.append(warehouse) 
            Tour_List.append(Tour)
    return Tour_List

Tour_List =  nearest_insertion(cluster_list)    


for i in Tour_List:   
    display(pd.DataFrame(i, columns = ["Nr.", "x", "y", "amount", "degree"]).iloc[:,0:-1])

        

,Nr.,x,y,amount
0,0,0.000000,0.000000,0.000000
1,20,37.793603,58.579721,9.126268
2,19,35.784464,51.781100,9.266826
3,6,54.391469,67.554083,5.464565
4,11,51.644713,49.482870,1.114538
5,0,0.000000,0.000000,0.000000


,Nr.,x,y,amount
0,0,0.000000,0.000000,0.000000
1,16,19.224185,35.510930,5.660820
2,5,38.975069,67.924465,8.137325
3,17,7.408676,70.093852,6.353562
4,15,5.209329,23.078673,8.166602
5,0,0.000000,0.000000,0.000000


,Nr.,x,y,amount
0,0,0.000000,0.000000,0.000000
1,2,-1.527402,4.287807,4.694884
2,13,-6.368330,29.297698,9.649153
3,10,-14.125190,41.515773,0.865257
4,21,4.382595,63.806373,8.248107
5,9,-3.427194,-8.013033,0.291116
6,8,7.480733,-26.037932,4.849311
7,0,0.000000,0.000000,0.000000


,Nr.,x,y,amount
0,0,0.000000,0.000000,0.000000
1,7,29.642482,-9.627123,7.708541
2,14,65.794386,-28.304371,6.317661
3,4,26.641802,-11.907302,3.989448
4,12,21.931605,-26.175279,2.512451
5,0,0.000000,0.000000,0.000000


,Nr.,x,y,amount
0,0,0.000000,0.000000,0.000000
1,3,69.661774,-14.052158,9.814687
2,1,64.873914,-3.109767,0.022142
3,18,40.679474,-2.558828,8.119024
4,0,0.000000,0.000000,0.000000
